### INSTALL REQUIRED LIBRARIES

In [1]:
!pip install pandas nltk scikit-learn keras tensorflow newspaper3k networkx
!pip install lxml_html_clean
!pip install newspaper3k

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 50.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.1/211.1 kB 19.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.3/81.3 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.4/107.4 kB 11.2 MB/s eta 0:00:00
  Created wheel for tinysegmenter: filename=tinysegmenter-0.3-py3-none-any.whl size=13540 sha256=81fd2e85be84365a295e090ec25ea122cf3ee3b6190f337aed05404e6f00a4d1
  Stored in directory: /root/.cache/pip/wheels/fc/ab/f8/cce3a9ae6d828bd346be695f7ff54612cd22b7cbd7208d68f3
  Created wheel for feedfinder2: filename=feedfinder2-0.0.4-py3-none-any.whl size=3341 sha256=a73868534f7133254510bf6b39aaae6e8a3848140a6062e2c6e34ae855e53fa0
  Stored in directory: /root/.cache/pip/wheels/80/d5/72/9cd9eccc819636436c6a6e59c22a0fb1e

### IMPORTS

In [2]:
import pandas as pd
import numpy as np
import nltk
import networkx as nx
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.stem import PorterStemmer

from keras.utils import to_categorical
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential, load_model
from keras.layers import Embedding, LSTM, Dense, Dropout
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from newspaper import Article
from sklearn.metrics.pairwise import cosine_similarity
nltk.download('punkt_tab')
nltk.download('punkt')
nltk.download('stopwords')
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


### TEXT CLEANING

In [3]:
stop_words = set(stopwords.words('english'))
stemmer = PorterStemmer()

def clean_text(text):
    text = str(text).lower()
    tokens = word_tokenize(text)
    tokens = [t for t in tokens if t.isalpha()]
    tokens = [t for t in tokens if t not in stop_words]
    tokens = [stemmer.stem(t) for t in tokens]
    return ' '.join(tokens)

### IMPORT DRIVE

In [4]:
from google.colab import drive
drive.mount('/content/drive')

MessageError: Error: credential propagation was unsuccessful

### LOAD & PROCESS DATA

In [ ]:
df = pd.read_csv("/content/drive/MyDrive/NLP/Project/Dataset/train.csv")
df.columns = ['class_id', 'title', 'description']

category_map = {
    1: 'World',
    2: 'Sports',
    3: 'Business',
    4: 'Sci/Tech'
}
df['category'] = df['class_id'].map(category_map)
df['text'] = df['title'].fillna('') + ". " + df['description'].fillna('')
df['cleaned'] = df['text'].apply(clean_text)

### TOKENIZATION

In [ ]:
max_words = 5000
max_len = 200

tokenizer = Tokenizer(num_words=max_words, oov_token="<OOV>")
tokenizer.fit_on_texts(df['cleaned'])
sequences = tokenizer.texts_to_sequences(df['cleaned'])
X = pad_sequences(sequences, maxlen=max_len)

### LABEL ENCODING

In [ ]:
le = LabelEncoder()
y = le.fit_transform(df['category'])
y = to_categorical(y)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

### LSTM MODEL

In [ ]:
model = Sequential()
model.add(Embedding(input_dim=max_words, output_dim=64, input_length=max_len))
model.add(LSTM(64, return_sequences=False))
model.add(Dropout(0.5))
model.add(Dense(32, activation='relu'))
model.add(Dense(4, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()


### TRAINING

In [ ]:
model.fit(X_train, y_train, epochs=5, batch_size=32, validation_split=0.2)

### EVALUATION

In [ ]:
loss, acc = model.evaluate(X_test, y_test)
print(f"\n✅ Test Accuracy: {acc:.2f}")

### SUMMARIZATION FUNCTION

In [ ]:
def summarize(text, top_n=2, max_words=40):
    sentences = sent_tokenize(text)
    if len(sentences) <= top_n:
        return ' '.join(sentences[:top_n])

    tfidf = Tokenizer()
    tfidf.fit_on_texts(sentences)
    tfidf_matrix = np.array([
        np.mean([tfidf.word_index.get(w, 0) for w in word_tokenize(s.lower()) if w.isalpha()], dtype=float)
        for s in sentences
    ]).reshape(-1, 1)

    sim_matrix = cosine_similarity(tfidf_matrix)
    nx_graph = nx.from_numpy_array(sim_matrix)
    scores = nx.pagerank(nx_graph)

    ranked = sorted(((scores[i], s, i) for i, s in enumerate(sentences)), reverse=True)

    summary = []
    total_words = 0
    for _, sentence, idx in sorted(ranked[:len(sentences)], key=lambda x: x[2]):
        word_count = len(sentence.split())
        if total_words + word_count <= max_words:
            summary.append(sentence)
            total_words += word_count
        if len(summary) >= top_n or total_words >= max_words:
            break

    return ' '.join(summary)


### URL FETCH FUNCTION

In [ ]:
def fetch_article(url):
    article = Article(url)
    article.download()
    article.parse()
    return article.title, article.text

### PREDICT FUNCTION

In [ ]:
def infer_news_category_and_summary_dl(url):
    title, text = fetch_article(url)
    cleaned = clean_text(title + ". " + text)
    seq = tokenizer.texts_to_sequences([cleaned])
    padded = pad_sequences(seq, maxlen=max_len)

    prediction = model.predict(padded)
    category = le.inverse_transform([np.argmax(prediction)])[0]
    summary = summarize(text)
    return {
        "title": title,
        "category": category,
        "summary": summary
    }

### TEST URL

In [ ]:
url = "https://www.thehindu.com/sci-tech/technology/why-is-google-buying-out-its-employees-explained/article69743598.ece"
result = infer_news_category_and_summary_dl(url)

print("\n📰 Title:\n", result['title'])
print("\n📂 Predicted Category:", result['category'])
print("\n📝 Summary:\n", result['summary'])
